### 전이 학습(Transfer Learning)

전이학습이란 기존의 잘 알려진 데이터 혹은 사전학습 된 모델을 업무 효율 증대나 도메인 확장을 위해 사용하는 학습을 의미한다. 따라서 전이학습은 인공지능 분야에서 매우 중요한 연구 중 하나이며 다양한 방법론들이 존재한다.

이미 학습된 모델의 지식을 새로운 문제에 활용한다. 이미 어떤 일을 잘하는 똑똑한 모델한테, 새로운 일을 좀 더 가르쳐 새로운 문제도 잘 풀게 학습시키는 맥락이다.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim

from tqdm import trange 

#### 1. GPU 연산 확인

빠른 병렬 처리를 위하여 CPU보다는 GPU연산이 사용된다. CUDA를 사용할 수 있는 NVIDIA 그래픽이 있다면 CUDA 버전과 torch 버전을 맞춰 본인의 컴퓨터를 세팅하면 된다.

In [3]:
# 현재 가능한 장치 확인
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(f'{device} is available.')

mps is available.


#### 2. CIFAR10 데이터 불러오기

In [ ]:
# Class
# 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

# 전처리 정의(transform)
# trainset
# 여러 이미지 전처리 과정을 순서대로 적용
train_transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4), # 32x32 크기로 이미지를 랜덤하게 자름, 잘리기 전에 각 가장자리에 4픽셀 씩 padding을 넣어줘 다양한 데이터가 생성 되게 함 (Data Augmentation)
     transforms.ToTensor(), # PIL 이미지 또는 numpy array를 Pytorch Tensor로 변환, 이미지 픽셀 값을 0~1 범위로 바꿔준다. (원래 각 픽셀은 0~255 사이의 정수값)
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # 채널(RGB)마다 평균 0.5, 표준 편차 0.5로 정규화 한다. -> 결과적으로 0~1 범위에서 -1~1 범위로 스케일이 변경된다. (대부분의 신경망은 입력값이 평균 0에 가까울 때 더 빠르고 안정적으로 학습된다.)

# testset 전용 전처리 정의
# 전처리 과정은 trainset과 동일하지만, 증강(RandomCrop) 없이 정규화만 한다.
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR-10 학습 데이터셋 불러오기
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
image, label = trainset[0]

print("Image shape:", image.shape) # CIFAR-10은 기본이 32X32여서, 별도로 resize 하지 않는 이상 -> Image shape: torch.Size([3, 32, 32]) => 채널 3개, 너비X높이 가 32*32 크기의 이미지

print("Label:", label) # Label: 6

# 학습용 데이터 로더 정의
# 데이터셋을 모델에 넣기 쉽게 배치 단위로 나눠준다.
# 매 epoch마다 데이터를 섞어서 학습에 편향이 없도록 한다.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
# 테스트셋은 평가용이기 때문에 섞지 않는다.
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

Image shape: torch.Size([3, 32, 32])
Label: 6


#### 3. Pretrained Model 불러오기
파이토치에서는 다양한 사전 학습 모델을 제공하고 있다.

In [5]:
# 사전 학습된 ResNet-18 불러오기
# weights='DEFAULT' : ResNet-18 구조를 쓰되, ImageNet으로 이미 똑똑해진 모델을 불러온다.
# weights=False를 하면 ResNet18 구조만 불러온다.
# 모델과 텐서에 .to(device)를 붙여야만 GPU 연산이 가능하다.

model = torchvision.models.resnet18(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/jinkyungjung/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 65.8MB/s]


In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### Model
```python
(fc): Linear(in_features=512, out_features=1000, bias=True)
```

in_features=512 -> 입력 차원이 512개. 즉, 이 레이어로 들어오는 feature 수가 512개 이다.
out_features=1000 -> 출력 차원이 1000개.

모델의 구조를 보면 마지막 레이어(fc = fully connected layer)의 출력 노드가 1000개라는 것을 알 수 있다.
이는 1000개의 클래스를 가진 ImageNet 데이터를 이용하여 사전 학습 된 모델이기 때문이다.
따라서 우리가 사용하는 CIFAR-10 데이터셋에 맞게 출력층의 노드를 10개로 변경해야 한다.

ResNet18의 기본 conv1은 (7x7 커널, stride=2)라서 너무 빨리 크기가 줄어들어버린다.(32x32 → 16x16 → 8x8 이렇게 너무 빨리 작아져) 

커널 사이즈를 7x7 ➔ 3x3 으로 줄이고, stride도 2 ➔ 1로 바꿔서, CIFAR-10처럼 작은 이미지를 더 잘 처리할 수 있게 맞춰준다.

##### Conv 레이어에서 크기 줄어드는 공식

\begin{equation}
  \Bigg[\frac{\text{Input Size}+2\times\text{Padding}-(\text{Kernel Size}-1)-1}{\text{Stride}}\Bigg]+1 \notag
\end{equation}


In [ ]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) # 이렇게 하면 feature 맵의 크기가 유지되어 정보 손실을 막을 수 있다.
model.fc = nn.Linear(512, 10) # 마지막 fully connected layer를 바꾸는 코드로, 마지막에 flatten 했을 때 나오는 feature 수인 512를, CIFAR-10 클래스 수인 10개로 바꿔준다.
model = model.to(device)

In [ ]:
print(model)

# input layer -> (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
# output layer -> (fc): Linear(in_features=512, out_features=10, bias=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### 4. 손실함수와 최적화 방법 정의

In [10]:
criterion = nn.CrossEntropyLoss() # 분류 문제에 적합한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2) # Adam 옵티마이저 사용, 학습률 0.0001, L2 정규화 0.01

In [ ]:
num_epochs = 20
ls = 2 # 현재까지의 최소 손실값 기록(loss saving)용 변수. 학습 초기에 큰 값 2로 설정
pbar = trange(num_epochs) # 진행률을 화면에 보여준다.

for epoch in pbar:
    correct = 0 # 맞춘 개수
    total = 0 # 전체 데이터 개수
    running_loss = 0.0 # 누적 손실값

    for data in trainloader: # 학습 데이터에서 미니 배치씩 거내온다.
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad() # 이전 배치에서 계산한 gradient을 초기화 함
        outputs = model(inputs) # 모델에 입력을 넣어서 예측값을 얻는다.
        loss = criterion(outputs, labels) # 손실함수를 계산해서 loss 계산
        loss.backward() # 오차를 역전파(모든 가중치에 대해 미분해서)해서 모델 가중치의 gradient 자동 계산
        optimizer.step() # 계산된 gradient을 이용해서 가중치 업데이트

        running_loss += loss.item()
        _, predicted = torch.max(outputs.detach(), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    cost = running_loss / len(trainloader) # 한 에폭 동안 평균 손실값(loss) 계산
    acc = 100*correct / total # 한 에폭 동안 평균 정확도

    if cost < ls:
        ls = cost
        torch.save(model.state_dict(), './models/cifar10_resnet18.pth')   

    pbar.set_postfix({'loss' : cost, 'train acc' : acc}) 


 65%|██████▌   | 13/20 [17:39<09:20, 80.09s/it, loss=2.45, train acc=10.9]

#### 모델 평가

In [ ]:
model = torchvision.models.resnet18(weights=None) # 사전 학습 없는 빈 ResNet18 모델
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) # CIFAR-10용으로 conv1 수정(3x3 커널), 출력층 수정(10개 클래스)
model.fc = nn.Linear(512, 10)
model = model.to(device)
model.load_state_dict(torch.load('./models/cifar10_resnet18.pth'))

#### 테스트 데이터 평가

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    model.eval()
    
    # 평가할 때 gradient 계산을 하지 않도록 한다.
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
# 85%